In [ ]:
import os
import sys
import json
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tqdm import tqdm
from collections import defaultdict, Counter

%matplotlib inline

project = 'AliAd'
path_to_raw = '../data/'+project+'/raw/'
path_to_processed = '../data/'+project+'/'

In [ ]:
# this cell is required for colab only

from google.colab import drive
drive.mount('/content/drive')

path_to_raw = './drive/MyDrive/Science/Multidomain RecSys/' + path_to_raw[3:]
path_to_processed = './drive/MyDrive/Science/Multidomain RecSys/source/' + path_to_processed[3:]

Mounted at /content/drive


In [ ]:
def create_all_data(df, output_path, output_name='all_data.txt', convert_dtypes=False):
    # part 1
    data = []
    for _, row in tqdm(df.iterrows()):
        data.append({
            'user_id': int(row.user_id) if convert_dtypes else row.user_id,
            'item_id':  int(row.item_id) if convert_dtypes else row.item_id,
            'timestamp': int(row.timestamp) if convert_dtypes else row.timestamp
        })


    # part 2
    user_history = defaultdict(list)
    item_history = defaultdict(list)

    for row in tqdm(data):
        user_raw_id = row['user_id']
        item_raw_id = row['item_id']
        interaction_timestamp = row['timestamp']

        user_history[user_raw_id].append({'item_id': item_raw_id, 'timestamp': interaction_timestamp})
        item_history[item_raw_id].append({'user_id': user_raw_id, 'timestamp': interaction_timestamp})


    # part 3
    is_changed = True
    threshold = 5
    good_users = set()
    good_items = set()

    while is_changed:
        old_state = (len(good_users), len(good_items))

        good_users = set()
        good_items = set()

        for user_id, history in user_history.items():
            if len(history) >= threshold:
                good_users.add(user_id)

        for item_id, history in item_history.items():
            if len(history) >= threshold:
                good_items.add(item_id)

        user_history = {
            user_id: list(filter(lambda x: x['item_id'] in good_items, history))
            for user_id, history in user_history.items()
        }

        item_history = {
            item_id: list(filter(lambda x: x['user_id'] in good_users, history))
            for item_id, history in item_history.items()
        }

        new_state = (len(good_users), len(good_items))
        is_changed = (old_state != new_state)
        print(old_state, new_state)


    # part 4
    user_mapping = {}
    item_mapping = {}
    tmp_user_history = defaultdict(list)
    tmp_item_history = defaultdict(list)

    for user_id, history in tqdm(user_history.items()):
        processed_history = []

        for filtered_item in history:
            item_id = filtered_item['item_id']
            item_timestamp = filtered_item['timestamp']

            processed_item_id = item_mapping.get(item_id, len(item_mapping) + 1)
            item_mapping[item_id] = processed_item_id

            processed_history.append({'item_id': processed_item_id, 'timestamp': item_timestamp})

        if len(processed_history) >= threshold:
            processed_user_id = user_mapping.get(user_id, len(user_mapping) + 1)
            user_mapping[user_id] = processed_user_id

            tmp_user_history[processed_user_id] = sorted(processed_history, key=lambda x: x['timestamp'])


    for item_id, history in tqdm(item_history.items()):
        processed_history = []

        for filtered_user in history:
            user_id = filtered_user['user_id']
            user_timestamp = filtered_user['timestamp']

            processed_user_id = user_mapping.get(user_id, len(user_mapping) + 1)
            user_mapping[user_id] = processed_user_id

            processed_history.append({'user_id': processed_user_id, 'timestamp': user_timestamp})

        if len(processed_history) >= threshold:
            processed_item_id = item_mapping.get(item_id, len(item_mapping) + 1)
            item_mapping[item_id] = processed_item_id

            tmp_item_history[processed_item_id] = sorted(processed_history, key=lambda x: x['timestamp'])

    user_history = tmp_user_history
    item_history = tmp_item_history

    # part 5
    with open(output_path+'/'+output_name, 'w') as f:
        for user_id, item_history in user_history.items():
            f.write(' '.join([str(user_id)] + [
                str(item_event['item_id']) for item_event in sorted(item_history, key=lambda x: x['timestamp'])
            ]))
            f.write('\n')

In [ ]:
files = [f for f in os.listdir(path_to_raw) if '.csv' in f]
domains = dict()
for f in files:
    k = f.split('_')[0]
    domains[k] = domains.get(k, []) + [f]

for d in tqdm(domains):
    new_dir = path_to_processed + d
    if not os.path.isdir(new_dir):
        os.makedirs(new_dir)

    files = []
    for f in domains[d]:
        df = pd.read_csv(path_to_raw+f, names = ['user_id', 'item_id', 'rating']).drop('rating', axis=1)
        files.append(df)

    df = pd.concat(files, axis=0)
    df['timestamp'] = pd.Series(range(1,df.shape[0]+1))

    _ = create_all_data(df, new_dir)

  0%|          | 0/8 [00:00<?, ?it/s]
0it [00:00, ?it/s]
2273it [00:00, 22721.52it/s]
4546it [00:00, 18164.02it/s]
6419it [00:00, 13658.38it/s]
7896it [00:00, 12356.32it/s]
9194it [00:00, 11086.56it/s]
10404it [00:00, 11345.28it/s]
11582it [00:00, 11430.64it/s]
12750it [00:01, 10480.11it/s]
13822it [00:01, 9091.12it/s] 
15001it [00:01, 9753.75it/s]
16228it [00:01, 10406.71it/s]
17379it [00:01, 10704.29it/s]
18482it [00:01, 10394.74it/s]
20492it [00:01, 11157.40it/s]

100%|██████████| 20492/20492 [00:00<00:00, 431214.52it/s]


(0, 0) (647, 846)
(647, 846) (361, 713)
(361, 713) (321, 566)
(321, 566) (269, 530)
(269, 530) (259, 493)
(259, 493) (251, 478)
(251, 478) (247, 473)
(247, 473) (245, 467)
(245, 467) (242, 466)
(242, 466) (242, 462)
(242, 462) (238, 462)
(238, 462) (238, 455)
(238, 455) (234, 455)
(234, 455) (234, 453)
(234, 453) (232, 453)
(232, 453) (232, 451)
(232, 451) (229, 451)
(229, 451) (229, 447)
(229, 447) (225, 447)
(225, 447) (225, 441)
(225, 441) (221, 441)
(221, 441) (221, 435)
(221, 435) (219, 435)
(219, 435) (219, 433)
(219, 433) (219, 433)



100%|██████████| 1702/1702 [00:00<00:00, 344931.65it/s]

 12%|█▎        | 1/8 [00:04<00:29,  4.16s/it]
0it [00:00, ?it/s]
2164it [00:00, 21638.22it/s]
4328it [00:00, 21582.53it/s]
6487it [00:00, 21438.67it/s]
8631it [00:00, 21156.79it/s]
10748it [00:00, 20712.00it/s]
12821it [00:00, 20168.71it/s]
14988it [00:00, 20641.05it/s]
17272it [00:00, 21324.80it/s]
19431it [00:00, 21405.10it/s]
21626it [00:01, 21569.92it/s]
23786it [00:01, 21063.15it/s]
25912it [00:01, 21120.66it/s]
28027it [00:01, 21036.25it/s]
30133it [00:01, 20171.15it/s]
32158it [00:01, 19998.39it/s]
34282it [00:01, 20357.98it/s]
36323it [00:01, 18056.78it/s]
38413it [00:01, 18824.63it/s]
40567it [00:01, 19581.85it/s]
42768it [00:02, 20273.09it/s]
44851it [00:02, 20432.80it/s]
47078it [00:02, 20968.11it/s]
49191it [00:02, 20215.84it/s]
51383it [00:02, 20705.16it/s]
53633it [00:02, 21227.52it/s]
55767it [00:02, 21028.15it/s]
57950it [00:02, 21262.47it/s]
60160it [00:02, 21509.41it/s]
62316it [00:03, 21237.58it/s]
64467it [00

(0, 0) (1031, 5365)
(1031, 5365) (958, 5300)
(958, 5300) (956, 5281)
(956, 5281) (956, 5280)
(956, 5280) (955, 5280)
(955, 5280) (955, 5280)



100%|██████████| 1768/1768 [00:00<00:00, 8616.20it/s]

 25%|██▌       | 2/8 [00:10<00:33,  5.59s/it]
0it [00:00, ?it/s]
1743it [00:00, 17427.27it/s]
3486it [00:00, 17175.12it/s]
5204it [00:00, 16055.37it/s]
6818it [00:00, 15745.96it/s]
8397it [00:00, 15546.20it/s]
9954it [00:00, 15234.50it/s]
11480it [00:00, 15071.59it/s]
12988it [00:00, 14457.86it/s]
15034it [00:00, 16235.51it/s]
17262it [00:01, 18031.57it/s]
19382it [00:01, 18975.72it/s]
21451it [00:01, 19486.39it/s]
23600it [00:01, 20083.94it/s]
25707it [00:01, 20378.32it/s]
27761it [00:01, 20425.55it/s]
29956it [00:01, 20880.47it/s]
32150it [00:01, 21197.21it/s]
34389it [00:01, 21553.96it/s]
36547it [00:01, 21096.22it/s]
38660it [00:02, 20988.39it/s]
40780it [00:02, 21049.19it/s]
43025it [00:02, 21462.85it/s]
45290it [00:02, 21814.90it/s]
47473it [00:02, 21735.01it/s]
49785it [00:02, 22145.95it/s]
52103it [00:02, 22452.30it/s]
54505it [00:02, 22920.32it/s]
56798it [00:02, 22312.43it/s]
59034it [00:02, 21494.09it/s]
61255it [00:03,

...


100%|██████████| 1901/1901 [00:00<00:00, 14700.33it/s]

 75%|███████▌  | 6/8 [00:30<00:09,  4.92s/it]
0it [00:00, ?it/s]
2349it [00:00, 23485.04it/s]
4698it [00:00, 21231.48it/s]
6851it [00:00, 21357.93it/s]
9075it [00:00, 21691.89it/s]
12565it [00:00, 21749.59it/s]

100%|██████████| 12565/12565 [00:00<00:00, 547513.19it/s]


(0, 0) (608, 548)
(608, 548) (288, 404)
(288, 404) (239, 260)
(239, 260) (178, 229)
(178, 229) (164, 177)
(164, 177) (126, 163)
(126, 163) (116, 128)
(116, 128) (90, 117)
(90, 117) (87, 95)
(87, 95) (67, 93)
(67, 93) (65, 76)
(65, 76) (54, 75)
(54, 75) (52, 63)
(52, 63) (48, 62)
(48, 62) (48, 58)
(48, 58) (45, 58)
(45, 58) (45, 57)
(45, 57) (45, 57)



100%|██████████| 1745/1745 [00:00<00:00, 507739.19it/s]

 88%|████████▊ | 7/8 [00:32<00:03,  3.85s/it]
0it [00:00, ?it/s]
2362it [00:00, 23613.38it/s]
4724it [00:00, 22691.63it/s]
6996it [00:00, 22042.68it/s]
9203it [00:00, 21410.48it/s]
11347it [00:00, 21331.44it/s]
13482it [00:00, 21167.65it/s]
15711it [00:00, 21522.83it/s]
19986it [00:00, 20466.72it/s]

100%|██████████| 19986/19986 [00:00<00:00, 796330.85it/s]


(0, 0) (687, 865)
(687, 865) (396, 717)
(396, 717) (357, 582)
(357, 582) (315, 548)
(315, 548) (306, 513)
(306, 513) (292, 508)
(292, 508) (289, 498)
(289, 498) (286, 496)
(286, 496) (285, 490)
(285, 490) (285, 490)



100%|██████████| 1679/1679 [00:00<00:00, 175915.18it/s]

100%|██████████| 8/8 [00:34<00:00,  4.35s/it]
